<h1>Titanic survival prediction</h1>

https://www.kaggle.com/c/titanic

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [30]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
train_data.select_dtypes(exclude=['object']).describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [32]:
train_data.select_dtypes(include=['object']).describe()

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


In [ ]:
#correration heatmap
plt.figure(figsize=(12, 12))
sns.heatmap(train_data.select_dtypes(exclude=["object"]).corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X = train_data.drop("Survived", axis=1)
y = train_data["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [33]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

class DoNothingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X
    def set_output(self, transform):
        return self

ordinalencoder = OrdinalEncoder()
donothingtransformer = DoNothingTransformer()

cols_encode = ["Sex", "Embarked"]
cols_nothing = ["SibSp", "Parch", "Age", "Fare", "Pclass"]

column_transformer = make_column_transformer(
    (ordinalencoder, cols_encode),
    (donothingtransformer, cols_nothing),
    remainder="drop"
)

pipe = make_pipeline(column_transformer, SimpleImputer(strategy="mean"))

In [41]:
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import GridSearchCV

ebm = ExplainableBoostingClassifier(random_state=42)

param_grid = {
    "learning_rate": [0.01, 0.1, 0.5, 1, 2],
}

processed_data = pipe.fit_transform(X_train)
grid_search = GridSearchCV(ebm, param_grid, cv=5, scoring="accuracy", return_train_score=True)

grid_search.fit(processed_data, y_train)

print(grid_search.best_params_)

{'learning_rate': 1}


In [43]:
grid_search.best_estimator_.score(pipe.transform(X_test), y_test)

0.7932960893854749

In [46]:
result = test_data[["PassengerId"]].copy()
result["Survived"] = grid_search.best_estimator_.predict(pipe.transform(test_data))
result.to_csv("submission.csv", index=False)